In [ ]:
ITERATORS

In [ ]:
import collections.abc as collections_abc
import typing

import utils.profilers as profiler_utils

T = typing.TypeVar("T")

In [1]:
import collections.abc as collections_abc
import typing

import utils.profilers as profiler_utils

T = typing.TypeVar("T")

In [3]:
SIZE = 10_000_000


def for_in_range() -> None:
    for _ in range(SIZE):
        ...

In [4]:
import functools
import time
import tracemalloc
import typing


def profile(func: typing.Callable) -> typing.Callable:
    @functools.wraps(func)
    def wrapper(*args: typing.Any, **kwargs: typing.Any) -> typing.Any:
        tracemalloc.start()
        start = time.time()
        result = func(*args, **kwargs)
        elapsed_time = (time.time() - start) * 1000
        _, mem_peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        print(
            f"{func.__name__}:\n max_memory: {mem_peak/1000_000:,.3f} mb\n exec time: {elapsed_time:,.3f} ms"
        )
        return result

    return wrapper

In [5]:
@profiler_utils.profile
def for_in_range() -> None:
    for _ in range(SIZE):
        ...

In [6]:
for_in_range()

for_in_range:
 max_memory: 0.001 mb
 exec time: 3,598.857 ms


In [7]:
@profiler_utils.profile
def run_through_iterable(items: collections_abc.Iterable[typing.Any]) -> None:
    for _ in items:
        ...

In [8]:
run_through_iterable(range(SIZE))

run_through_iterable:
 max_memory: 0.001 mb
 exec time: 3,588.717 ms


In [9]:
def range_generator() -> collections_abc.Iterator[int]:
    yield from range(SIZE)

In [10]:
def list_generator() -> list[int]:
    return list(range(SIZE))

In [11]:
for_in_range()

for_in_range:
 max_memory: 0.001 mb
 exec time: 3,633.248 ms


In [12]:
gen1 = range_generator()
run_through_iterable(gen1)

run_through_iterable:
 max_memory: 0.006 mb
 exec time: 3,905.991 ms


In [13]:
gen2 = list_generator()
run_through_iterable(gen2)

run_through_iterable:
 max_memory: 0.006 mb
 exec time: 82.593 ms


In [14]:
@profiler_utils.profile
def range_generator() -> collections_abc.Iterator[int]:
    yield from range(SIZE)

In [15]:
@profiler_utils.profile
def list_generator() -> list[int]:
    return list(range(SIZE))

In [16]:
for_in_range()

for_in_range:
 max_memory: 0.005 mb
 exec time: 3,681.350 ms


In [17]:
gen1 = range_generator()
run_through_iterable(gen1)

range_generator:
 max_memory: 0.000 mb
 exec time: 0.008 ms
run_through_iterable:
 max_memory: 0.007 mb
 exec time: 4,220.290 ms


In [18]:
gen2 = list_generator()
run_through_iterable(gen2)

list_generator:
 max_memory: 359.997 mb
 exec time: 3,938.272 ms
run_through_iterable:
 max_memory: 0.002 mb
 exec time: 71.386 ms


In [19]:
def yield_from_layer(items: collections_abc.Iterable[T]) -> collections_abc.Iterator[T]:
    yield from items

In [20]:
def yield_for_layer(items: collections_abc.Iterable[T]) -> collections_abc.Iterator[T]:
    for item in items:
        yield item

In [21]:
def expression_layer(items: collections_abc.Iterable[T]) -> collections_abc.Iterator[T]:
    return (item for item in items)

In [22]:
for_in_range()

for_in_range:
 max_memory: 0.002 mb
 exec time: 3,686.507 ms


In [23]:
run_through_iterable(
    yield_from_layer(yield_from_layer(yield_from_layer(yield_from_layer(yield_from_layer(range(SIZE))))))
)

run_through_iterable:
 max_memory: 0.006 mb
 exec time: 5,527.732 ms


In [24]:
run_through_iterable(
    yield_for_layer(yield_for_layer(yield_for_layer(yield_for_layer(yield_for_layer(range(SIZE))))))
)

run_through_iterable:
 max_memory: 0.005 mb
 exec time: 5,574.007 ms


In [25]:
run_through_iterable(
    expression_layer(expression_layer(expression_layer(expression_layer(expression_layer(range(SIZE)))))),
)


run_through_iterable:
 max_memory: 0.004 mb
 exec time: 5,430.608 ms
